# Preparation of data
This section will consist to exatct and transform data to be ready to upload in data warehouse


In [ ]:
# Import all needed packages or modules
from pyspark.sql import SparkSession
from datetime import datetime, date
import pandas as pd

In [ ]:
# Creation of spark Session
spark = SparkSession.builder.getOrCreate()